<a href="https://colab.research.google.com/github/Sindhiya08/Account-Registration-Login-using-Python-and-File-handling/blob/main/NLP_MCMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing the required packages for the project

Transformers -> To generate question and answers from the pretrained model

Pdf Miner-> To extract the text content from PDF

In [1]:
!pip install transformers[sentencepiece]
!pip install pdfminer.six
!pip install pypdf



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 5.1 MB/s eta 0:00:00


cloning the github repo to access the pdf file

In [2]:
! git clone https://github.com/akaiketech/internship-assignment-nlp

Cloning into 'internship-assignment-nlp'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 18 (delta 5), reused 5 (delta 2), pack-reused 0
Receiving objects: 100% (18/18), 8.94 MiB | 23.42 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [3]:
#Importing the req pre trained model from transformer along with the Pdf extracter

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from pdfminer.high_level import extract_text

# Loading tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("allenai/macaw-large")
model = AutoModelForSeq2SeqLM.from_pretrained("allenai/macaw-large")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Using the Macaw model from the transformers to generate Q&A



In [ ]:

# the pdf file will be an input passed in the function
def mcma_questions(pdf_path):

  # this will check whether the file provided is in string format or not,If not raises an error.
    if not isinstance(pdf_path, str):
      raise ValueError("'pdf_path' should be a string.")



    # Extracts text from PDF which will be stored in the variable
    text = extract_text(pdf_path)

    # Setting up the input string with placeholders(here we are using 4 placeholders)
    # context- the extratcted text will be passed in to this as a passage to generate Question,Answer,and Options
    input_string = "$answer$ ; $mcoptions$ ; $question$ ; $context$ "
    input_text = input_string.replace("$context$", text)

    # Tokenize and encode the input text, Encoded input will be stored in input_ids
    # have set the maximum length, the input will be taken till the set max_length
    # inputs will be truncated once the set max length is exceeded
    # By setting return_tensors="pt", the tokenizer will return the encoded inputs as PyTorch tensors instead of regular Python lists for further processing.

    inputs = tokenizer.encode_plus(input_text, max_length=5000, truncation=True, return_tensors="pt")
    input_ids = inputs["input_ids"]

    # taking the inputs,output will be generated.
    # Num_return_sequences and num_beams - defines the number of outputs required to generate accoridngly (here we will get 20 questions in output) and beam search width
    # tensors will be converted to string again ,conversion will happen in batch and skipping the special characters

    output = model.generate(input_ids, max_length=5000, num_return_sequences=20, num_beams=20)
    decoded_outputs = tokenizer.batch_decode(output, skip_special_tokens=True)

    return decoded_outputs

# path to PDF file from which needs to be extracted

pdf_path = "/content/internship-assignment-nlp/Dataset/chapter-2.pdf"
questions_output = mcma_questions(pdf_path)

# Iterating the output to split the questions,answers and MCoptions which will be stored in a empty list as tuples by replacing the placeholder text.
# Printing the Questions along with its answer and MC options

mcma_ques = []
for out in questions_output:
    components = out.split(";")
    if len(components) >= 3:
        question = components[2].strip().replace("$question$ =", "")
        options = [option.strip().replace("$mcoptions$ =", "") for option in components[1].split(",")]
        answers = [answer.strip().replace("$answer$ =", "") for answer in components[0].split(",")]
        mcma_ques.append((question, options, answers))
    else:
        print(f"Invalid format encountered: {out}")

for i, (question, options, answers) in enumerate(mcma_ques, 1):
    print(f" {i}.  Question: {question}")
    for j, option in enumerate(options, 1):
        print(f" Options: {option}")
    print(f" Answer: {', '.join(answers)}")
    print()
